<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load libraries</a></span></li><li><span><a href="#Prepare-the-dataset" data-toc-modified-id="Prepare-the-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare the dataset</a></span></li></ul></div>

# Load libraries

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Prepare the dataset

In [2]:
data_dir = "./data/"

In [3]:
audio_pairs_df = pd.read_csv(data_dir + "audio_sentence_pairs_test.csv")
audio_pairs_df

,student_I_id,article_I_id,sentence_I_id,student_II_id,article_II_id,sentence_II_id
0,18020017_TranManhHieu,1,17,18020017_TranManhHieu,1,6
1,17021332_LeMinhTam,7,26,17021298_NguyenDinhNhatMinh,1,27
2,18020833_NguyenKimLong,8,3,18020833_NguyenKimLong,8,4
3,17021332_LeMinhTam,4,11,18020833_NguyenKimLong,2,22
4,17021311_CaoMinhNhat,1,3,17021281_NguyenTrongLam,7,11
...,...,...,...,...,...,...
19995,17021281_NguyenTrongLam,4,11,17021332_LeMinhTam,1,28
19996,17021311_CaoMinhNhat,8,2,17021311_CaoMinhNhat,6,15
19997,18020833_NguyenKimLong,3,14,17021281_NguyenTrongLam,5,0
19998,17021298_NguyenDinhNhatMinh,4,14,17021281_NguyenTrongLam,7,18


In [4]:
import _pickle as cPickle
import preprocessing

TEST_SET = ["val", "test"]
VAL_OR_TEST = 1
MFCC_LEN_LIM = 700

waveform_dir = data_dir + "waveform/"

def create_mfcc_data(student_id, article_id, sentence_id):
    audio_sentence = np.load(waveform_dir + student_id + "/{:02d}/".format(article_id + 1) + str(sentence_id) + ".npy")
    return preprocessing.mfcc(audio_sentence)[:MFCC_LEN_LIM]

data_mfcc = {
    "sentence_I": [],
    "sentence_II": [],
    "same_speaker": []
}

audio_pairs = audio_pairs_df.to_numpy()[VAL_OR_TEST * len(audio_pairs_df) // 2:
                                        (VAL_OR_TEST+1) * len(audio_pairs_df) // 2]
print('Progress: ', end='')
for i in range(len(audio_pairs)):
    row = audio_pairs[i]
    data_mfcc['sentence_I'].append(create_mfcc_data(row[0], row[1], row[2]))
    data_mfcc['sentence_II'].append(create_mfcc_data(row[3], row[4], row[5]))
    data_mfcc['same_speaker'].append(row[0] == row[3])
    if i * 100 // len(audio_pairs) > (i - 1) * 100 // len(audio_pairs):
        print('|', end='')
    
cPickle.dump(data_mfcc, open(data_dir + f"dataset/data_mfcc_{TEST_SET[VAL_OR_TEST]}.pkl", 'wb'), protocol=2)

Progress: ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||